## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Paamiut,GL,61.99,-49.67,37.89,92,100,58.54,moderate rain
1,1,Chifeng,CN,42.27,118.96,52.47,44,0,19.19,clear sky
2,2,Bluff,NZ,-46.60,168.33,57.00,57,100,11.01,light rain
3,3,Bud,US,39.45,-86.18,54.00,62,75,8.05,broken clouds
4,4,Changde,CN,29.05,111.68,71.06,91,94,4.99,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Changde,CN,29.05,111.68,71.06,91,94,4.99,overcast clouds
5,5,Arraial Do Cabo,BR,-22.97,-42.03,74.46,85,0,25.79,clear sky
6,6,Carnarvon,AU,-24.87,113.63,68.00,68,75,11.41,broken clouds
9,9,Kapaa,US,22.08,-159.32,86.00,62,1,9.17,clear sky
11,11,Bengkulu,ID,-3.80,102.27,77.00,90,99,9.26,overcast clouds
12,12,Cabo San Lucas,MX,22.89,-109.91,89.60,70,5,17.22,clear sky
13,13,Namibe,AO,-15.20,12.15,60.98,87,18,2.51,few clouds
15,15,Rikitea,PF,-23.12,-134.97,73.49,79,0,18.88,clear sky
16,16,Lagoa,PT,39.05,-27.98,68.23,85,100,14.97,overcast clouds
17,17,Guerrero Negro,MX,27.98,-114.06,83.35,41,0,14.85,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                406
City                   406
Country                405
Lat                    406
Lng                    406
Max Temp               406
Humidity               406
Cloudiness             406
Wind Speed             406
Current Description    406
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Changde,CN,71.06,overcast clouds,29.05,111.68,
5,Arraial Do Cabo,BR,74.46,clear sky,-22.97,-42.03,
6,Carnarvon,AU,68.00,broken clouds,-24.87,113.63,
9,Kapaa,US,86.00,clear sky,22.08,-159.32,
11,Bengkulu,ID,77.00,overcast clouds,-3.80,102.27,
12,Cabo San Lucas,MX,89.60,clear sky,22.89,-109.91,
13,Namibe,AO,60.98,few clouds,-15.20,12.15,
15,Rikitea,PF,73.49,clear sky,-23.12,-134.97,
16,Lagoa,PT,68.23,overcast clouds,39.05,-27.98,
17,Guerrero Negro,MX,83.35,clear sky,27.98,-114.06,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df=hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(25.0, 101.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))